# 예제 3.52: 체크포인트(Checkpoint) 불러오기

## 학습목표
1. **저장된 체크포인트 불러오기** 방법 익히기
2. **딕셔너리에서 각 항목 추출** 방법 이해하기
3. **학습 재개(Resume Training)** 구현하기
4. **옵티마이저 상태 복원** 의 중요성 이해하기

---

#### 라이브러리 및 클래스 정의

**학습 재개 시 옵티마이저 상태 복원이 중요한 이유**
- SGD with momentum: 이전 기울기 정보 유지
- Adam: 1차/2차 모멘트 추정치 유지
- 상태 미복원 시 학습이 처음부터 시작하는 효과

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    """커스텀 데이터셋 클래스"""
    
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length


class CustomModel(nn.Module):
    """커스텀 모델 클래스"""
    
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

---

#### 데이터 및 모델 준비

In [ ]:
# 데이터셋 및 데이터로더 생성
train_dataset = CustomDataset("../datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

In [ ]:
# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델, 손실함수, 옵티마이저 생성 (빈 상태)
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

---

#### 체크포인트 불러오기

딕셔너리 키를 사용해 각 항목을 추출하고 복원

In [ ]:
# 체크포인트 불러오기 (6번째 체크포인트 = 6000 에포크)
checkpoint = torch.load("../models/checkpoint-6.pt")

# 모델 파라미터 복원
model.load_state_dict(checkpoint["model_state_dict"])

# 옵티마이저 상태 복원 (모멘텀 등 내부 상태)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

# 에포크 정보 추출 (학습 재개 시작점)
checkpoint_epoch = checkpoint["epoch"]

# 설명 출력
checkpoint_description = checkpoint["description"]
print(f"불러온 체크포인트: {checkpoint_description}")
print(f"재개 시작 에포크: {checkpoint_epoch + 1}")

---

#### 학습 재개 (Resume Training)

체크포인트 에포크 다음부터 학습 계속

In [ ]:
# 체크포인트 이후부터 학습 재개
# range(checkpoint_epoch + 1, 10000): 6001~9999 에포크 학습
for epoch in range(checkpoint_epoch + 1, 10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)
    
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")